# KIX 4F Simulation (yokhr) - 32 SBDs with CU airlines

## Imports and Initializations

In [ ]:
%load_ext autoreload
%autoreload 2

from copy import deepcopy

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from src.utils.simparam import SimParam
from src.simfunc.simulation import Simulation
from src.utils.optimizer import custmin
from src.utils.utils import day_graph, minutes_to_hms

%matplotlib inline
%config InlineBackend.figure_format='retina'

## FY2019 Schedule & T1 6kPAX Schedule

In [ ]:
# Get 6kPAX schedule
simparam6k = SimParam()
path_to_6k_sched = (
    r"/home/yokhr/projects/KAPpaxsim/data/processed/Schedule (30th terminal peak, 6000"
    r" pax)_PROCESSED_yokhr.xlsx"
)
simparam6k.schedule_from_path(path_to_6k_sched)
simparam6k.schedule_cleanup().schedule_filter().assign_check_in()
simparam6k.show_up_from_file().assign_flight_show_up_category_default().assign_show_up()

# Get FY2019 data
data = pd.read_excel(
    io="/home/yokhr/projects/KAPpaxsim/data/raw/FY2019_FY2025 merge.xlsx",
    header=0,
    sheet_name="IntlP_FY19-FY25",
)

# Format the file
mask2019 = (data["FY"] == "FY2019") & (data["Day Of Week"] == "Saturday")
data2019 = data[mask2019].reset_index().copy()
data2019["Sector"] = data2019["Int/Dom"]
data2019["Flight Date"] = pd.to_datetime("2019-10-13")
data2019["Scheduled Time"] = data2019["5min Interval"].apply(
    lambda x: f"{x // 100}:{x % 100}:00",
)
data2019["Category(P/C/O)"] = data2019["Category(P/C/O)"].apply(lambda x: x[0])

# Create simparam2019
simparam2019 = SimParam()
simparam2019.schedule_from_df(data2019).schedule_cleanup().schedule_filter(
    date_str="2019-10-13"
).assign_check_in()
simparam2019.show_up_from_file().assign_flight_show_up_category_default().assign_show_up()

# Plot
simparam2019.plot_name = "FY19"
simparam6k.plot_name = "6kPAX"
simparam2019.plot_std()
simparam2019.plot_counters()

## Select ALs + Set Parameters

In [ ]:
selected_ALs = [
    "AF",
    "KL",
    "NH",
    "JL",
    "CX",
    "EK",
    "OZ",
    "UA",
    "AY",
    "SQ",
    "KE",
    "CA",
    "BA",
    "QF",
    "VN",
]

RNm_kiosk = 92  # Resource number of Kiosk
RNm_CUSBD_ls = [32]  # Resource number of CUSBD
Ptm_kiosk = 70  # Process time of Kiosk
Ptm_CUSBD_ls = [60]  # Process time of CUSBD
Ptm_CkIn = 80  # Process time of CheckIn
HrSTD_CUSBD = 2.5  # Hours to STD of CUSBD
HrSTD_CkIn = 2.5  # Hours to STD of CheckIn

## Plot Graph & Processing Time Simulation in FY19

In [ ]:
airline_CU_2019 = []

for Ptm_CUSBD in Ptm_CUSBD_ls:
    for RNm_CUSBD in RNm_CUSBD_ls:
        for i in range(len(selected_ALs)):
            airline_CU_2019.append(selected_ALs[i])

            # Set CU airlines & Tradi airlines
            simparam = simparam2019
            simparam_tradi = deepcopy(simparam)

            all_airlines = simparam.df_Counters.iloc[:, :-1].sum(axis=0).index.to_list()
            airlines_tradi = [x for x in all_airlines if x not in airline_CU_2019]
            mask_airlines_CU = simparam.schedule["Airline Code"].isin(airline_CU_2019)
            simparam_tradi.schedule = simparam_tradi.schedule[
                mask_airlines_CU == False
            ].copy()
            simparam_tradi.assign_check_in()

            # Plot check-in graph
            print(f"N [CU airlines]: {len(airline_CU_2019)} {airline_CU_2019}")
            print(f"N [Tradi airlines]: {len(airlines_tradi)} {sorted(airlines_tradi)}")
            print(f"MAX Tradi Counters: {simparam_tradi.df_Counters['total'].max()}")
            simparam_tradi.plot_name = f"{len(airlines_tradi)} Tradi airlines"
            simparam_tradi.plot_counters(compare_with=simparam)

            # Define PaxType
            simparam2019.df_Pax["pax_type"] = simparam2019.df_Pax["Airline"].apply(
                lambda x: "CUSBD" if x in airline_CU_2019 else "tradi"
            )
            simparam2019.schedule["pax_type"] = simparam2019.schedule[
                "Airline Code"
            ].apply(lambda x: "CUSBD" if x in airline_CU_2019 else "tradi")
            simparam2019.plot_std(by_col=True, col_name="pax_type")

            # Set Parameters
            simparam2019.dct_resource = {
                "kiosk": RNm_kiosk,
                "CUSBD": RNm_CUSBD,
            }
            simparam2019.dct_processes = {
                "kiosk": Ptm_kiosk,
                "CUSBD": Ptm_CUSBD,
                "checkin": Ptm_CkIn,
            }
            simparam2019.dct_process_sequence = {
                "tradi": [
                    ["kiosk", "wait_opening", "checkin"],
                    {"hour_to_std": HrSTD_CkIn},
                ],
                "CUSBD": [
                    ["kiosk", "wait_opening", "CUSBD"],
                    {"hour_to_std": HrSTD_CUSBD},
                ],
            }

            # Print Parameter
            print(f"The number of CUSBD: {RNm_CUSBD}")
            print(f"Processing Time [CUSBD]: {Ptm_CUSBD} sec")
            print(f"Opening hrs before STD [CUSBD]: {HrSTD_CUSBD} hr")

            # Processing Simulation
            simulation2019 = Simulation(simparam2019)
            simulation2019.generate_checkin().generate_pax().run()
            simulation2019.format_df_result()

            # Print Top 90% pax waiting time
            wtCUSBD = round(simulation2019.dct_hist_wait_time["CUSBD"].quantile(0.9), 2)
            print(f"Top 90% waiting time [CUSBD]: {wtCUSBD} min")
            wtCkIn = round(
                simulation2019.dct_hist_wait_time["checkin"].quantile(0.9), 2
            )
            print(f"Top 90% waiting time [CkIn]: {wtCkIn} min")

            # Plot graph
            simulation2019.plot_result()